In [ ]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip
! pip install -q efficientnet
!pip install -q -U tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import efficientnet.tfkeras as efn

In [ ]:
%rm -rf ./train_images
shutil.copytree("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train","./images")

In [ ]:
train_df = pd.read_csv("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv")
classes = list(train_df["Class"].unique())
classes_dict = {}
for i,label in enumerate(classes):
    classes_dict[label] = i
    
train_df.index = train_df["Image"]
train_df=train_df.drop(["Image"],axis=1)
train_df.head()

print(classes_dict)
print(train_df.head())

In [ ]:
train_path = "./images"
train_ls = os.listdir(train_path)

if os.path.exists("./train") != True:
    os.mkdir("./train")
    for i in range(6):
        os.mkdir("./train/"+str(i))
        
for name in train_ls:
    path = os.path.join(train_path,name)
    label = classes_dict[train_df.loc[name]["Class"]]
    final_path = "./train/"+str(label)
    shutil.move(path,final_path)

In [ ]:
HEIGHT= 300
WIDTH = 300
BATCH_SIZE = 32
NUM_IMAGES = len(train_ls)
SPLIT = 0.85
CLASSES = 6

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split = 1-SPLIT,
        rotation_range=25,
        fill_mode="nearest",
        height_shift_range = 0.15,
        width_shift_range = 0.15,
)

train_generator = train_datagen.flow_from_directory(
    "./train",
    target_size = (HEIGHT,WIDTH),
    batch_size=BATCH_SIZE,
    class_mode = "categorical",
    shuffle = True,
    subset = "training"
)


validation_generator = train_datagen.flow_from_directory(
    "./train",
    target_size = (HEIGHT,WIDTH),
    batch_size=BATCH_SIZE,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation"
)

In [ ]:
def create_model():
    efficient_net = {
        0 : efn.EfficientNetB0(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        1 : efn.EfficientNetB1(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        2 : efn.EfficientNetB2(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        3 : efn.EfficientNetB3(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        4 : efn.EfficientNetB4(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        5 : efn.EfficientNetB5(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        6 : efn.EfficientNetB6(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        7 : efn.EfficientNetB7(weights="noisy-student",include_top=False ,input_shape=[HEIGHT,WIDTH, 3]),
        8 : tf.keras.applications.Xception(include_top=False, weights='imagenet',input_shape=[HEIGHT,WIDTH, 3]),
        9 : tf.keras.applications.ResNet50(include_top=False, weights='imagenet',input_shape=[HEIGHT,WIDTH, 3]),
        10: tf.keras.applications.ResNet101(include_top=False, weights='imagenet',input_shape=[HEIGHT,WIDTH, 3]),
        11: tf.keras.applications.ResNet152(include_top=False, weights='imagenet',input_shape=[HEIGHT,WIDTH, 3])
    }

    output = {}
    inputs = tf.keras.Input(shape=(HEIGHT,WIDTH, 3))
    
    ls =   [5]   
    
    
    for i in ls:
        pretrained_model = efficient_net[i]
        pretrained_model.trainable = False
        x = pretrained_model(inputs)
        x = tf.keras.layers.GlobalAveragePooling2D(name = "average_"+str(i))(x)
        x = tf.keras.layers.Dense(512,activation="relu")(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        x = tf.keras.layers.Dense(256,activation="relu")(x)
        output[i] = tf.keras.layers.Dense(CLASSES,activation="softmax", dtype='float32',name="dense_"+str(i))(x)
    
    if len(ls)>1:
        outputs = tf.keras.layers.average(list(output.values()))
    else:
        outputs = list(output.values())[0]
        
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
import tensorflow_addons as tfa

def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
        
    metrics = [
       tfa.metrics.F1Score(num_classes=CLASSES, average = "weighted",name="f1_score")
    ]
   
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
def create_callbacks():
    
    cpk_path = './best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor='val_f1_score',
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_f1_score',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_f1_score',
        mode='max',
        patience=15, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks

In [ ]:
EPOCHS = 50

with tf.device('/device:GPU:0'):
    model = create_model()
    model = compile_model(model, lr=0.0001)
   
    callbacks = create_callbacks()
    
    history = model.fit(train_generator,
                    epochs= EPOCHS,
                    verbose=1,
                    validation_data = validation_generator,
                    callbacks= callbacks
                       )

In [ ]:
acc = history.history['f1_score']
val_acc = history.history['val_f1_score']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(history.history['val_loss']))
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training F1 Score')
plt.plot(epochs_range, val_acc, label='Validation F1 Score')
plt.legend(loc='lower right')
plt.title('Training and Validation F1 Score')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model = tf.keras.models.load_model("./best_model.h5")

In [ ]:
test_path = "../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/"
test_ls = os.listdir(test_path)
predictions=[]

label_dict = {
    0:'Miscellaneous', 1:'Candle', 2:'Snowman', 3:'Airplane', 4:'Christmas_Tree', 5:'Jacket'
}

for filename in test_ls:
    img = tf.keras.preprocessing.image.load_img(
      test_path+filename,target_size = (HEIGHT,WIDTH)
    )
    arr = tf.keras.preprocessing.image.img_to_array(img)
    arr = tf.expand_dims(arr/255.,0)
    predictions.append(label_dict[np.argmax(model.predict(arr)[0])])

df = pd.DataFrame(zip(test_ls,predictions),columns = ["Image","Class"])
df.to_csv("./submission.csv",index=False)
df.head()

In [ ]:
%rm -rf ./images
%rm -rf ./train